In [19]:
import glob
import itertools
from pydub import AudioSegment

In [8]:
reciters_paths = glob.glob('./reciters/*/*')

In [25]:
paths_dictionary = {}
for i, j in itertools.groupby(reciters_paths, key=lambda x: x.split('/')[-2]):
    paths_dictionary[i] = list(j)

In [33]:
reciter_path = paths_dictionary['1']

In [34]:
path = reciter_path[0]

In [35]:
sound = AudioSegment.from_mp3(path)

In [37]:
sound.frame_rate

44100

In [36]:
for i, chunk in enumerate(sound[::5000]):
    print(type(chunk.get_array_of_samples()))

<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'array.array'>
<class 'ar